## Import lbraries.
---
This cell calls the libraries needed for computation, for downloading the data, and for plotting it.

In [ ]:
# Numerical libraries:
import numpy as np
import pandas as pd
from scipy import interpolate
from scipy.optimize import curve_fit

#SunPy libraries (for CDAWeb):
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.timeseries import TimeSeries

#For handling timestamps:
import datetime as dt
from datetime import timedelta

#Plotting libraries:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import ticker